## **<div style="text-align:center"><span style="font-size:1em;"> <code>Segmentez des clients d'un site e-commerce</code></span> </div>**

- Vous êtes consultant pour **Olist**, une entreprise brésilienne qui propose une solution de vente sur **les marketplaces en ligne**.
  
- Votre rôle est d’accompagner Olist dans leur projet de monter une équipe Data et leur premier cas d’usage Data Science autour de **la segmentation client.**

Ce notebook contient: 

- =>  l'analyse l'**EDA** (**E**xploratory **D**ata **A**nalysis)

- =>  la **création du dataset** OLIST

- =>  le **nettoyage du dataset** OLIST

- =>  le calcul de la **RFM (Recency - Frequency  - Monetary)** à savoir : 1. **Recence** (**nb. de jour** depuis la dernière cmd) 2. **Frequency** (**Nb. Total de cmd**) 3. **Monetary** (**le montant total des commandes** pour chaque client)

- =>  La préparation **(standardisation)* des données** qui nous serviront à entrainer les modèles de Machine Learning, modèles d'apprentissage **non supervisés**

_*Transformer les données pour qu'elles aient une moyenne de 0 et un écart-type de 1._


# ETAPE #1 - Présentation | Compréhension du jeu de données OLIST

In [ ]:
# Importation des librairies necessaires a - l'Exploratory Data Analysis - (EDA) sur la base de donnees d'OLIST
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import zscore

# Chargement des datasets (les fichiers csv qui constituent le jeu de donnees d'OLIST)
datasets = {
    "Geolocalisation": pd.read_csv('olist_geolocation_dataset.csv'),
    "Customers_Olist": pd.read_csv('olist_customers_dataset.csv'),
    "Products_Olist": pd.read_csv('olist_products_dataset.csv'),
    "Items_Olist": pd.read_csv('olist_order_items_dataset.csv'),
    "Orders_Olist": pd.read_csv('olist_orders_dataset.csv'),
    "Orders_Payments_Olist": pd.read_csv('olist_order_payments_dataset.csv'),
    "Orders_Review_Olist": pd.read_csv('olist_order_reviews_dataset.csv'),
    "Sellers_Olist": pd.read_csv('olist_sellers_dataset.csv'),
    "Product_Category_Name_Translated_Olist": pd.read_csv('product_category_name_translation.csv')
}

# La fonction pour effectuer une analyse exploratoire complete
def eda_complet(df, dataset_name):
    """
    Effectue une analyse exploratoire complete.

    Args:
        df (pd.DataFrame): Le DataFrame a analyser.
        dataset_name (str): Le nom du dataset pour le contexte des sorties.

    Returns:
        None
    """
    print(f"\n{'='*40}\nAnalyse du dataset : {dataset_name}\n{'='*40}\n")
    
    # Apercu des donnees
    print("Premieres 5 lignes :\n", df.head(), "\n")
    print("Dimensions (lignes, colonnes) :", df.shape, "\n")
    print("Informations sur les colonnes :")
    print(df.info(), "\n")
    
    # Valeurs manquantes
    print("Valeurs manquantes :")
    valeurs_manquantes = df.isnull().sum()
    print(valeurs_manquantes[valeurs_manquantes > 0], "\n")
    msno.matrix(df)
    plt.title(f"Valeurs manquantes dans {dataset_name}")
    plt.show()
    
    # Statistiques descriptives
    print("Statistiques descriptives :\n", df.describe(include='all'), "\n")
    
    # Lignes dupliquees
    doublons = df.duplicated().sum()
    print(f"Nombre de lignes dupliquees : {doublons}\n")
    
    # Analyse des colonnes numeriques
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print("Statistiques descriptives des colonnes numeriques :\n", df[numeric_cols].describe().T, "\n")
        
        # Distributions
        for col in numeric_cols:
            plt.figure(figsize=(10, 5))
            sns.histplot(df[col], kde=True, bins=30)
            plt.title(f"Distribution de {col}")
            plt.xlabel(col)
            plt.ylabel("Frequence")
            plt.show()
        
        # Valeurs aberrantes
        z_scores = np.abs(zscore(df[numeric_cols].dropna()))
        outliers = (z_scores > 3).sum(axis=0)
        print("Valeurs aberrantes detectees par colonne :\n", outliers, "\n")
        
        # Heatmap de correlation
        if len(numeric_cols) > 1:
            plt.figure(figsize=(12, 8))
            sns.heatmap(df[numeric_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
            plt.title(f"Heatmap des correlations : {dataset_name}")
            plt.show()
    
    # Analyse des colonnes categorielles
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"Colonnes categoricielles detectees : {list(categorical_cols)}\n")
        for col in categorical_cols:
            print(f"Top valeurs pour {col} :\n", df[col].value_counts().head(5), "\n")
            plt.figure(figsize=(10, 5))
            sns.countplot(data=df, y=col, order=df[col].value_counts().index[:10], palette="viridis")
            plt.title(f"Distribution de {col}")
            plt.xlabel("Frequence")
            plt.ylabel(col)
            plt.show()

    print(f" Fin de l'analyse pour {dataset_name}\n")

# Effectuer l'analyse exploratoire pour chaque dataset
for nom, df in datasets.items():
    try:
        eda_complet(df, nom)
    except Exception as e:
        print(f"Erreur lors de l'analyse du dataset {nom} : {e}")


# ETAPE #2 - Transformer les données | Création du dataset pour la segmentation des clients RFM

In [2]:
# Connaître l'emplacement du fichier référent à la base de données OLIST
import os

# Obtenir le répertoire courant
current_directory = os.getcwd()

# Afficher le répertoire courant
print(f"Répertoire courant : {current_directory}")


Répertoire courant : C:\Python311\Scripts\NOUVEAU P5


In [3]:
import sqlite3
import pandas as pd
import os

# Chemin d'accès au fichier contenant la base de données OLIST ext.SQLite
chemin_fichier = "C:/Python311/Scripts/NOUVEAU P5/olist.db"

def jointure_dataset(chemin_fichier):
    try:
        # Vérification de l'existence du fichier
        if not os.path.exists(chemin_fichier):
            print(f"Erreur : Le fichier '{chemin_fichier}' n'existe pas.")
            return

        # Connexion à la base de données SQLite
        connexion = sqlite3.connect(chemin_fichier)
        print("Connexion réussie à la base de données.\n")

        # Étape 1 : Chargement des tables nécessaires et connexion aux tables
        print(
            "Chargement des tables : 'order_items', 'orders', 'products', "
            "'customers' et 'order_reviews'..."
        )
        order_items = pd.read_sql_query("SELECT * FROM order_items", connexion)
        orders = pd.read_sql_query("SELECT * FROM orders", connexion)
        products = pd.read_sql_query("SELECT * FROM products", connexion)
        customers = pd.read_sql_query("SELECT * FROM customers", connexion)
        order_reviews = pd.read_sql_query("SELECT * FROM order_reviews", connexion)
        print("Les tables sont maintenant chargées avec succès.\n")

        # Étape 2 : Suppression des colonnes 'index' si elles existent
        print("Suppression des colonnes redondantes (index)")
        order_items = order_items.drop(columns=['index'], errors='ignore')
        orders = orders.drop(columns=['index'], errors='ignore')
        products = products.drop(columns=['index'], errors='ignore')
        customers = customers.drop(columns=['index'], errors='ignore')
        order_reviews = order_reviews.drop(columns=['index'], errors='ignore')

        # Étape 3 : Vérification des colonnes des tables
        print("\nVérification des colonnes des tables :")
        print("Colonnes de 'order_items' :", order_items.columns)
        print("Colonnes de 'orders' :", orders.columns)
        print("Colonnes de 'products' :", products.columns)
        print("Colonnes de 'customers' :", customers.columns)
        print("Colonnes de 'order_reviews' :", order_reviews.columns)

        # Étape 4 : Vérification des correspondances clés entre les tables
        print("\nVérification des correspondances clés :")
        print(f"Nombre de 'order_id' uniques dans 'order_items' : {order_items['order_id'].nunique()}")
        print(f"Nombre de 'order_id' uniques dans 'orders' : {orders['order_id'].nunique()}")
        print(f"Nombre de 'product_id' uniques dans 'products' : {products['product_id'].nunique()}")
        print(f"Nombre de 'customer_id' uniques dans 'customers' : {customers['customer_id'].nunique()}")

        # Étape 5 : Jointure 'order_items' avec 'orders'
        print("\nJointure de 'order_items' avec 'orders' sur 'order_id'...")
        order_items_orders = pd.merge(order_items, orders, on='order_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'orders' : {len(order_items_orders)}")

        # Étape 6 : Jointure avec 'products'
        print("\nJointure avec 'products' sur 'product_id'...")
        orders_products = pd.merge(order_items_orders, products, on='product_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'products' : {len(orders_products)}")

        # Étape 7 : Jointure avec 'customers'
        print("\nJointure avec 'customers' sur 'customer_id'...")
        orders_products_customers = pd.merge(orders_products, customers, on='customer_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'customers' : {len(orders_products_customers)}")

        # Étape 8 : Jointure avec 'order_reviews' pour ajouter 'review_score'
        print("\nJointure avec 'order_reviews' sur 'order_id' pour ajouter 'review_score'...")
        dataset_final = pd.merge(
            orders_products_customers,
            order_reviews[['order_id', 'review_score']],
            on='order_id',
            how='left'
        )
        print(f"Nombre de lignes après la jointure avec 'order_reviews' : {len(dataset_final)}")

        # Étape 9 : Affichage des colonnes finales et aperçu du dataset
        print("\nColonnes finales dans le dataset obtenu :")
        print(dataset_final.columns)

        print("\nAperçu des données finales :")
        print(dataset_final.head())

        # Étape 10 : Exportation du dataset final en CSV
        output_file = "jointure_complete_avec_review_score.csv"
        dataset_final.to_csv(output_file, index=False)
        print(f"\nLe fichier CSV '{output_file}' a été créé avec succès dans le répertoire courant.")

        # Fermeture de la connexion
        connexion.close()

    except sqlite3.Error as e:
        print(f"Erreur lors de la lecture de la base de données : {e}")
    except Exception as ex:
        print(f"Une erreur inattendue est survenue : {ex}")


# Appel de la fonction
jointure_dataset(chemin_fichier)


Connexion réussie à la base de données.

Chargement des tables : 'order_items', 'orders', 'products', 'customers' et 'order_reviews'...
Les tables sont maintenant chargées avec succès.

Suppression des colonnes redondantes (index)

Vérification des colonnes des tables :
Colonnes de 'order_items' : Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
Colonnes de 'orders' : Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Colonnes de 'products' : Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Colonnes de 'customers' : Index(['cus

In [4]:
import sqlite3
import pandas as pd
import os

# Chemin d'accès au fichier contenant la base de données OLIST ext.SQLite
chemin_fichier = "C:/Python311/Scripts/NOUVEAU P5/olist.db"

def jointure_dataset(chemin_fichier):
    try:
        # Vérification de l'existence du fichier
        if not os.path.exists(chemin_fichier):
            print(f"Erreur : Le fichier '{chemin_fichier}' n'existe pas.")
            return

        # Connexion à la base de données SQLite
        connexion = sqlite3.connect(chemin_fichier)
        print("Connexion réussie à la base de données.\n")

        # Étape 1 : Chargement des tables nécessaires et connexion aux tables
        print(
            "Chargement des tables : 'order_items', 'orders', 'products', "
            "'customers' et 'order_reviews'..."
        )
        order_items = pd.read_sql_query("SELECT * FROM order_items", connexion)
        orders = pd.read_sql_query("SELECT * FROM orders", connexion)
        products = pd.read_sql_query("SELECT * FROM products", connexion)
        customers = pd.read_sql_query("SELECT * FROM customers", connexion)
        order_reviews = pd.read_sql_query("SELECT * FROM order_reviews", connexion)
        print("Les tables sont maintenant chargées avec succès.\n")

        # Étape 2 : Suppression des colonnes 'index' si elles existent
        print("Suppression des colonnes redondantes (index)...")
        order_items = order_items.drop(columns=['index'], errors='ignore')
        orders = orders.drop(columns=['index'], errors='ignore')
        products = products.drop(columns=['index'], errors='ignore')
        customers = customers.drop(columns=['index'], errors='ignore')
        order_reviews = order_reviews.drop(columns=['index'], errors='ignore')

        # Étape 3 : Vérification des colonnes des tables
        print("\nVérification des colonnes des tables :")
        print("Colonnes de 'order_items' :", order_items.columns)
        print("Colonnes de 'orders' :", orders.columns)
        print("Colonnes de 'products' :", products.columns)
        print("Colonnes de 'customers' :", customers.columns)
        print("Colonnes de 'order_reviews' :", order_reviews.columns)

        # Étape 4 : Vérification des correspondances clés entre les tables
        print("\nVérification des correspondances clés :")
        print(f"Nombre de 'order_id' uniques dans 'order_items' : {order_items['order_id'].nunique()}")
        print(f"Nombre de 'order_id' uniques dans 'orders' : {orders['order_id'].nunique()}")
        print(f"Nombre de 'product_id' uniques dans 'products' : {products['product_id'].nunique()}")
        print(f"Nombre de 'customer_id' uniques dans 'customers' : {customers['customer_id'].nunique()}")

        # Étape 5 : Jointure 'order_items' avec 'orders'
        print("\nJointure de 'order_items' avec 'orders' sur 'order_id'...")
        order_items_orders = pd.merge(order_items, orders, on='order_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'orders' : {len(order_items_orders)}")

        # Étape 6 : Jointure avec 'products'
        print("\nJointure avec 'products' sur 'product_id'...")
        orders_products = pd.merge(order_items_orders, products, on='product_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'products' : {len(orders_products)}")

        # Étape 7 : Jointure avec 'customers'
        print("\nJointure avec 'customers' sur 'customer_id'...")
        orders_products_customers = pd.merge(orders_products, customers, on='customer_id', how='inner')
        print(f"Nombre de lignes après la jointure avec 'customers' : {len(orders_products_customers)}")

        # Étape 8 : Jointure avec 'order_reviews' pour ajouter 'review_score'
        print("\nJointure avec 'order_reviews' sur 'order_id' pour ajouter 'review_score'...")
        dataset_final = pd.merge(
            orders_products_customers,
            order_reviews[['order_id', 'review_score']],
            on='order_id',
            how='left'
        )
        print(f"Nombre de lignes après la jointure avec 'order_reviews' : {len(dataset_final)}")

        # Étape 9 : Affichage des colonnes finales et aperçu du dataset
        print("\nColonnes finales dans le dataset obtenu :")
        print(dataset_final.columns)

        print("\nAperçu des données finales :")
        print(dataset_final.head())

        # Étape 10 : Exportation du dataset final en CSV
        output_file = "jointure_complete_avec_review_score.csv"
        dataset_final.to_csv(output_file, index=False)
        print(f"\nLe fichier CSV '{output_file}' a été créé avec succès dans le répertoire courant.")

        # Fermeture de la connexion
        connexion.close()

    except sqlite3.Error as e:
        print(f"Erreur lors de la lecture de la base de données : {e}")
    except Exception as ex:
        print(f"Une erreur inattendue est survenue : {ex}")


# Appel de la fonction
jointure_dataset(chemin_fichier)


Connexion réussie à la base de données.

Chargement des tables : 'order_items', 'orders', 'products', 'customers' et 'order_reviews'...
Les tables sont maintenant chargées avec succès.

Suppression des colonnes redondantes (index)...

Vérification des colonnes des tables :
Colonnes de 'order_items' : Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
Colonnes de 'orders' : Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Colonnes de 'products' : Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Colonnes de 'customers' : Index(['

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Nom du fichier généré par le script précédent
fichier_csv = "jointure_complete_avec_review_score.csv"

def eda_sur_dataset(fichier_csv):
    try:
        # Chargement des données
        print("\nChargement du fichier CSV pour l'EDA...")
        df = pd.read_csv(fichier_csv)
        print("Le dataset est chargé")
        print(f"Nombre de lignes : {df.shape[0]}, Nombre de colonnes : {df.shape[1]}")

        # Aperçu des premières lignes
        print("\nAperçu des premières lignes :")
        print(df.head())

        # Informations générales sur le dataset
        print("\nInformations générales sur le dataset :")
        print(df.info())

        # Résumé statistique
        print("\nRésumé statistique des colonnes numériques :")
        print(df.describe())

        # Vérification des données manquantes
        print("\nAnalyse des données manquantes :")
        missing_values = df.isnull().sum()
        print(missing_values[missing_values > 0])

        # Distribution des variables numériques
        print("\nCréation des visualisations des variables numériques...")
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        df[numeric_columns].hist(bins=15, figsize=(15, 10), edgecolor='black')
        plt.suptitle("Distributions des variables numériques", fontsize=16)
        plt.show()

        # Analyse de la variable `order_status`
        if 'order_status' in df.columns:
            print("\nDistribution de la variable 'order_status' :")
            order_status_counts = df['order_status'].value_counts()
            print(order_status_counts)

            plt.figure(figsize=(10, 6))
            sns.countplot(data=df, x='order_status', order=order_status_counts.index)
            plt.title("Distribution de 'order_status'")
            plt.xlabel("Statut de la commande")
            plt.ylabel("Nombre")
            plt.xticks(rotation=45)
            plt.show()

        # Analyse des relations entre quelques variables clés
        if {'price', 'freight_value'}.issubset(df.columns):
            print("\nAnalyse de la relation entre 'price' et 'freight_value' :")
            plt.figure(figsize=(8, 6))
            sns.scatterplot(data=df, x='price', y='freight_value')
            plt.title("Relation entre 'price' et 'freight_value'")
            plt.xlabel("Prix")
            plt.ylabel("Valeur du fret")
            plt.show()

        if {'price', 'product_category_name'}.issubset(df.columns):
            print("\nAnalyse des prix par catégorie de produit :")
            avg_price_by_category = df.groupby('product_category_name')['price'].mean().sort_values(ascending=False)
            print(avg_price_by_category.head())

            plt.figure(figsize=(10, 8))
            avg_price_by_category.plot(kind='bar')
            plt.title("Prix moyen par catégorie de produit")
            plt.xlabel("Catégorie de produit")
            plt.ylabel("Prix moyen")
            plt.xticks(rotation=90)
            plt.show()

        # Corrélation entre les variables numériques
        print("\nAnalyse des corrélations entre les variables numériques :")
        correlation_matrix = df[numeric_columns].corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Matrice de corrélation")
        plt.show()

        print("\nEDA terminé avec succès.")
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{fichier_csv}' est introuvable.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")

# Appel de la fonction pour effectuer l'EDA
eda_sur_dataset(fichier_csv)


In [5]:
# Lecture du dataset d'OLIST
# Le fichier 'jointure_complete_avec_review_score.csv' contient les données jointes et consolidées.

OLIST_merged_data = pd.read_csv('jointure_complete_avec_review_score.csv')


In [6]:
# Afficher les premières lignes du dataset
# Cette commande permet de visualiser un aperçu des 5 premières lignes du dataset
# pour vérifier son contenu et sa structure.

OLIST_merged_data.head()


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,4.0,650.0,28.0,9.0,14.0,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,5.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,2.0,30000.0,50.0,30.0,40.0,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,4.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,2.0,3050.0,33.0,13.0,33.0,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,5.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,1.0,200.0,16.0,10.0,15.0,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,1.0,3750.0,35.0,40.0,30.0,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,5.0


In [7]:
# Afficher le résumé statistique du dataset
# La méthode describe() génère des statistiques descriptives pour les colonnes numériques du dataset,
# telles que la moyenne, l'écart type, les valeurs minimales, maximales, et les percentiles.

OLIST_merged_data.describe()


,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,review_score
count,113314.000000,113314.000000,113314.000000,111702.000000,111702.000000,111702.000000,113296.000000,113296.000000,113296.000000,113296.000000,113314.000000,112372.000000
mean,1.198528,120.478701,19.979428,48.777560,786.899250,2.206908,2091.915037,30.162495,16.584513,23.003539,35122.306670,4.032473
std,0.707016,183.279678,15.783227,10.024616,651.758866,1.719500,3749.804597,16.151737,13.439206,11.708481,29869.796752,1.387849
min,1.000000,0.850000,0.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1003.000000,1.000000
25%,1.000000,39.900000,13.080000,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000,11310.000000,4.000000
50%,1.000000,74.900000,16.260000,52.000000,601.000000,1.000000,700.000000,25.000000,13.000000,20.000000,24340.000000,5.000000
75%,1.000000,134.900000,21.150000,57.000000,985.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,59041.500000,5.000000
max,21.000000,6735.000000,409.680000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,99990.000000,5.000000


In [9]:
# Affichage des colonnes du dataset
# La propriété .columns retourne une liste des noms de colonnes du dataset.
# Cela permet de vérifier la structure du dataframe et les noms des champs disponibles.

OLIST_merged_data.columns


Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'review_score'],
      dtype='object')

# ETAPE #2 - Transformer les données | Nettoyage du dataset

In [ ]:
# Nettoyage du dataset avant de passer au calcul de la RFM (Recency / Frequency / Monetary)

import pandas as pd

# Étape 1 : Aperçu initial du dataset
print("Aperçu des données avant nettoyage :")
print(OLIST_merged_data.info())  # Affiche les informations générales du dataframe, y compris les valeurs manquantes
print(OLIST_merged_data.head())  # Affiche un aperçu des 5 premières lignes du dataframe

# Vérification et affichage des valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(OLIST_merged_data.isnull().sum())

# Étape 2 : Gestion des anomalies
# Suppression des lignes avec des prix négatifs ou égaux à 0
OLIST_merged_data = OLIST_merged_data[OLIST_merged_data['price'] > 0]

# Étape 3 : Vérification et conversion des types de données
# Conversion des colonnes contenant des dates en format datetime
colonnes_dates = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'shipping_limit_date'
]

for col in colonnes_dates:
    OLIST_merged_data[col] = pd.to_datetime(OLIST_merged_data[col], errors='coerce')

# Étape 4 : Vérification finale après nettoyage
print("\nAperçu des données après nettoyage :")
print(OLIST_merged_data.info())  # Vérifie les modifications dans les types de données et les valeurs manquantes
print(OLIST_merged_data.head())  # Affiche les 5 premières lignes du dataframe nettoyé

# Vérification des valeurs manquantes après le nettoyage
print("\nValeurs manquantes après nettoyage :")
print(OLIST_merged_data.isnull().sum())


# ETAPE #2 - Transformer les données | Définition de la R.F.M Recency Frequency Monetary pour le criblage Marketing


- La segmentation RFM ou méthode RFM est une méthode de segmentation principalement développée à l'origine pour les actions de marketing direct des véadistes* et qui s'applique désormais également aux acteurs du e-commerce et du commerce traditionnel. _*Entreprises spécialisées dans la vente à distance._

- La segmentation RFM prend en compte la **Récence (date de la dernière commande)**, la **Fréquence des commandes** et le **Montant (de la dernière commande ou sur une période donnée)** pour établir des segments de clients homogènes.

- La segmentation RFM permet de cibler les offres, d'établir des segments basés sur la valeur des clients et de prévenir l'attrition en identifiant des segments à risque.

- Les données RFM restent des données très utilisées, mais une "simple" segmentation RFM est de plus en plus rare. **Les données RFM sont désormais intégrées dans des procédures de ciblage et de segmentation comportant de plus en plus de variables et de données**. Elles dépassent désormais le cadre du marketing direct et peuvent s'appliquer à la publicité display en utilisant une DMP et des procédures de CRM onboarding.

Dans cette étude de cas on définit les variables RFM (Recency, Frequency, Monetary) comme suivant :
- **Récence** : Le nombre de jours depuis la dernière commande.
- **Fréquence** : Le nombre de commandes passées par le client.
- **Montant** : Le total des montants dépensés par le client.

# ETAPE #2 - Transformer les données | **RECENCE** (Nb. de jour TOTAL depuis la dernière commande par Client)

In [ ]:
import pandas as pd

# Conversion explicite de 'order_purchase_timestamp' en type datetime
# Cette étape garantit que la colonne est bien interprétée comme une série temporelle.
OLIST_merged_data['order_purchase_timestamp'] = pd.to_datetime(
    OLIST_merged_data['order_purchase_timestamp']
)

# Définition d'une date de référence
# La date de référence est définie comme la date la plus récente dans les données.
reference_date = OLIST_merged_data['order_purchase_timestamp'].max()
print(f"Date de référence pour le calcul de la récence : {reference_date}")

# Calcul de la récence
# Pour chaque client unique ('customer_unique_id'), on calcule la récence (nombre de jours)
# en soustrayant la dernière date d'achat de la date de référence.
OLIST_merged_data['recency'] = OLIST_merged_data.groupby('customer_unique_id')[
    'order_purchase_timestamp'
].transform(lambda x: (reference_date - x.max()).days)

# Aperçu du dataset après ajout de la colonne 'recency'
print("Aperçu du dataset après ajout de la colonne 'recency' :")
print(OLIST_merged_data.head())


In [9]:
# Calcul de la moyenne de la colonne 'recency'
# La moyenne est calculée pour évaluer la récence moyenne des clients
# (nombre moyen de jours depuis la dernière interaction client).
recency_mean = OLIST_merged_data['recency'].mean()

# Affichage de la moyenne
# La valeur calculée est affichée avec un message explicite.
print(f"La moyenne de recency est : {recency_mean}")


La moyenne de recency est : 242.04252784298498


In [ ]:
# Afficher la liste des colonnes du dataframe
# Cette commande permet de vérifier la structure du dataframe et de s'assurer
# que la colonne calculée 'recency' a bien été ajoutée.
print("Liste des colonnes du dataframe :")
print(OLIST_merged_data.columns)


# ETAPE #2 - Transformer les données | **FREQUENCE** (Nb. TOTAL de Commande par Client)

In [ ]:
import pandas as pd

# Calcul de la fréquence
# La fréquence représente le nombre total de commandes passées par chaque client unique.
# Cette valeur est calculée en regroupant par 'customer_unique_id' et en comptant le nombre
# de 'order_id' pour chaque client. Le résultat est ajouté sous forme de nouvelle colonne.
OLIST_merged_data['frequency'] = OLIST_merged_data.groupby('customer_unique_id')[
    'order_id'
].transform('count')

# Aperçu du dataset après ajout de la colonne 'frequency'
# Affichage des premières lignes du dataset pour vérifier l'ajout et la cohérence de la colonne.
print("Aperçu du dataset après ajout de la colonne 'frequency' :")
print(OLIST_merged_data.head())


In [ ]:
# Calcul de la moyenne de la colonne 'frequency'
# La moyenne est calculée pour déterminer le nombre moyen de commandes passées
# par client sur l'ensemble du dataset.
frequency_mean = OLIST_merged_data['frequency'].mean()

# Affichage de la moyenne
# La moyenne calculée est affichée avec un message explicatif pour plus de clarté.
print(f"La moyenne de la fréquence est : {frequency_mean}")


In [ ]:
# Afficher la liste des colonnes du dataframe
# Cette commande permet de vérifier si la colonne calculée 'frequency' a bien été ajoutée
# au dataframe principal. Elle affiche la liste complète des noms de colonnes.
print("Liste des colonnes du dataframe :")
print(OLIST_merged_data.columns)


# ETAPE #2 - Transformer les données | **MONTANT** (Montant total des commandes passées par client)

In [ ]:
# Calcul du montant total des commandes par client
# La variable 'monetary' représente le montant total des achats effectués par chaque client unique.
# Pour cela, on regroupe les données par 'customer_unique_id' et on calcule la somme des prix ('price')
# pour chaque client. Le résultat est ajouté au dataframe principal sous forme d'une nouvelle colonne.
OLIST_merged_data['monetary'] = OLIST_merged_data.groupby('customer_unique_id')[
    'price'
].transform('sum')

# Aperçu des données après ajout de la colonne 'monetary'
# Affichage des premières lignes du dataset pour vérifier l'ajout et la cohérence de la colonne 'monetary'.
print("Aperçu du dataset avec la colonne 'monetary' :")
print(OLIST_merged_data.head())


In [ ]:
# Calcul de la moyenne de la colonne 'monetary'
# La moyenne est calculée pour analyser le montant moyen des achats effectués
# par client sur l'ensemble du dataset.
monetary_mean = OLIST_merged_data['monetary'].mean()

# Affichage de la moyenne
# La valeur calculée est affichée 
print(f"La moyenne de monetary est : {monetary_mean}")


La moyenne de monetary est : 167.09293555959547


In [ ]:
# Afficher la liste des colonnes du dataframe
# Cette commande permet de vérifier si la colonne calculée 'monetary' a bien été ajoutée
# au dataframe principal. Elle affiche la liste complète des noms de colonnes disponibles.
print("Liste des colonnes du dataframe :")
print(OLIST_merged_data.columns)


In [ ]:
# Afficher le nombre de valeurs nulles par colonne
# Cette commande permet de vérifier la quantité de valeurs manquantes dans chaque colonne
# du dataframe. Ces informations sont essentielles pour combler ou traiter les valeurs
# nulles avant d'implémenter des modèles de machine learning non supervisés.
print("Nombre de valeurs nulles par colonne :")
print(OLIST_merged_data.isnull().sum())


# ETAPE #2 - Transformer les données | Traitement du **review score** | Standardisation des données (mise à l'échelle des données)

- La standardisation permet de centrer les valeurs autour de 0 avec un écart-type de 1, rendant la variable recency comparable à d'autres variables pour les modèles sensibles aux échelles comme **_K-means_** et **_DBSCAN_**.

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Étape 1 : Remplacement des valeurs nulles dans 'review_score' par la moyenne
print("\nRemplacement des valeurs nulles dans 'review_score' par la moyenne...")

# Calcul du nombre total de valeurs et du nombre de valeurs manquantes
nombre_total_valeurs = OLIST_merged_data['review_score'].shape[0]
nombre_valeurs_manquantes = OLIST_merged_data['review_score'].isnull().sum()

# Calcul de la moyenne et remplacement des valeurs manquantes
review_score_mean = OLIST_merged_data['review_score'].mean()
OLIST_merged_data['review_score'] = OLIST_merged_data['review_score'].fillna(review_score_mean)

# Affichage des statistiques
print(f"Nombre total de valeurs dans 'review_score' : {nombre_total_valeurs}")
print(f"Nombre de valeurs remplacées : {nombre_valeurs_manquantes}")
print(f"Proportion de valeurs remplacées : {nombre_valeurs_manquantes / nombre_total_valeurs:.2%}")
print(f"Moyenne utilisée pour 'review_score' : {review_score_mean}")

# Étape 2 : Encodage One-Hot de 'review_score'
print("\nEncodage de la colonne 'review_score' avec One-Hot Encoding...")
one_hot_encoder = OneHotEncoder(sparse_output=False)

# Transformer 'review_score' en variables One-Hot
review_score_encoded = one_hot_encoder.fit_transform(OLIST_merged_data[['review_score']])

# Récupérer les noms des nouvelles colonnes après encodage
review_score_columns = [f"review_score_{int(cat)}" for cat in one_hot_encoder.categories_[0]]
print(f"Nouvelles colonnes créées : {review_score_columns}")

# Ajouter les colonnes encodées au DataFrame
OLIST_merged_data = OLIST_merged_data.drop(columns=['review_score'])  # Supprimer l'ancienne colonne
OLIST_merged_data[review_score_columns] = review_score_encoded


In [ ]:
# Créer une copie du DataFrame
# Cette étape est utile pour préserver le DataFrame original avant d'effectuer des modifications.
OLIST_merged_data_copy = OLIST_merged_data.copy()

# Exporter la copie si nécessaire
# La copie est exportée en fichier CSV pour des usages futurs ou pour sauvegarder l'état actuel des données.
OLIST_merged_data_copy.to_csv("OLIST_merged_data_copy.csv", index=False)

# Réaffecter la copie au DataFrame d'origine pour les prochaines opérations
# Cela permet d'utiliser la version sauvegardée pour les prochaines transformations ou analyses.
OLIST_merged_data = OLIST_merged_data_copy

# Vérifier les colonnes après la réaffectation
# Affiche la liste des colonnes pour confirmer que le DataFrame conserve sa structure attendue.
print("Liste des colonnes après réaffectation :")
print(OLIST_merged_data.columns)


In [ ]:
import matplotlib.pyplot as plt

# Vérification de la distribution de la colonne 'recency'
# Création d'un histogramme pour visualiser la répartition des valeurs de récence (en jours)
plt.hist(OLIST_merged_data['recency'], bins=30, edgecolor='black')
plt.title("Distribution de la colonne 'recency'")
plt.xlabel("Recency (jours)")
plt.ylabel("Nombre d'occurrences")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Résumé statistique de la colonne 'recency'
# Fournit des statistiques descriptives comme la moyenne, les quartiles, le min et max.
print("Résumé statistique de la colonne 'recency' :")
print(OLIST_merged_data['recency'].describe())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialisation du scaler StandardScaler
# Le StandardScaler standardise les données en les centrant sur la moyenne (0)
# et en les échelonnant selon l'écart type (1).
scaler = StandardScaler()

# Standardisation de la colonne 'recency'
# La méthode fit_transform ajuste le scaler sur les données et les transforme.
OLIST_merged_data['recency'] = scaler.fit_transform(OLIST_merged_data[['recency']])

# Vérification des résultats après standardisation (moyenne de 0 et un écart-type de 1.)
print("\nRésumé statistique de la colonne 'recency' après normalisation :")
print(OLIST_merged_data['recency'].describe())

# Aperçu des premières valeurs standardisées
print("\nAperçu des premières lignes de la colonne 'recency' après standardisation :")
print(OLIST_merged_data[['recency']].head())


In [ ]:
import matplotlib.pyplot as plt

# Vérification de la distribution de la colonne 'frequency'
# Création d'un histogramme pour visualiser la répartition des valeurs de fréquence
# (nombre total de commandes par client unique).
plt.hist(OLIST_merged_data['frequency'], bins=30, edgecolor='black')
plt.title("Distribution de la colonne 'frequency'")
plt.xlabel("Frequency (nombre de commandes par client)")
plt.ylabel("Count (nombre d'occurrences)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Résumé statistique de la colonne 'frequency'
# Affiche des statistiques descriptives comme la moyenne, les quartiles, le min et max.
print("Résumé statistique de la colonne 'frequency' :")
print(OLIST_merged_data['frequency'].describe())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialisation du scaler StandardScaler
# Le StandardScaler standardise les données en les centrant sur la moyenne (0)
# et en les échelonnant selon l'écart type (1).
scaler = StandardScaler()

# Standardisation de la colonne 'frequency'
# La méthode fit_transform ajuste le scaler sur les données et les transforme en même temps.
OLIST_merged_data['frequency'] = scaler.fit_transform(OLIST_merged_data[['frequency']])

# Vérification des résultats après standardisation
print("\nRésumé statistique de la colonne 'frequency' après standardisation :")
print(OLIST_merged_data['frequency'].describe())

# Aperçu des premières valeurs standardisées
print("\nAperçu des premières lignes de la colonne 'frequency' après standardisation :")
print(OLIST_merged_data[['frequency']].head())


In [ ]:
import matplotlib.pyplot as plt

# Vérification de la distribution de la colonne 'monetary'
# Création d'un histogramme pour visualiser la répartition des valeurs de la colonne 'monetary'
# (montant total des achats par client).
plt.hist(OLIST_merged_data['monetary'], bins=30, edgecolor='black')
plt.title("Distribution de la colonne 'monetary'")
plt.xlabel("Monetary (montant total des achats)")
plt.ylabel("Count (nombre d'occurrences)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Résumé statistique de la colonne 'monetary'
# Fournit des statistiques descriptives comme la moyenne, les quartiles, le min et max.
print("Résumé statistique de la colonne 'monetary' :")
print(OLIST_merged_data['monetary'].describe())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialisation du StandardScaler
# Le StandardScaler standardise les données en les centrant sur la moyenne (0)
# et en les échelonnant selon l'écart type (1).
scaler = StandardScaler()

# Standardisation de la colonne 'monetary'
# Transformation des valeurs pour qu'elles soient standardisées.
OLIST_merged_data['monetary'] = scaler.fit_transform(OLIST_merged_data[['monetary']])

# Vérification des résultats après standardisation
print("\nRésumé statistique de la colonne 'monetary' après standardisation :")
print(OLIST_merged_data['monetary'].describe())

# Aperçu des premières lignes de la colonne standardisée
print("\nAperçu des premières lignes de la colonne 'monetary' après standardisation :")
print(OLIST_merged_data[['monetary']].head())


In [ ]:
import pandas as pd

# Lecture du dataset
# Chargement des données préparées pour les modèles de machine learning
# depuis un fichier CSV nommé 'dataset_prepared_for_ml.csv'.
OLIST_merged_data = pd.read_csv('dataset_prepared_for_ml.csv')

# Affichage des premières lignes pour vérification
# Cette étape permet de visualiser un aperçu des données pour confirmer
# que le chargement s'est effectué correctement.
print("Aperçu des premières lignes du dataset :")
print(OLIST_merged_data.head())

# Affichage des informations sur le dataset
# Affiche des détails comme le nombre de colonnes, les types de données,
# les valeurs non nulles, et la mémoire utilisée par le dataset.
print("\nInformations sur le dataset :")
print(OLIST_merged_data.info())


In [ ]:
# Afficher les variances des colonnes 'recency', 'frequency', et 'monetary'
# La variance mesure la dispersion des valeurs autour de leur moyenne.
# Une variance élevée indique une dispersion importante, tandis qu'une variance faible indique que les valeurs sont proches de la moyenne.

print("Variance de 'recency' :", OLIST_merged_data['recency'].var())
print("Variance de 'frequency' :", OLIST_merged_data['frequency'].var())
print("Variance de 'monetary' :", OLIST_merged_data['monetary'].var())


In [ ]:
# Affichage des résumés statistiques des colonnes sélectionnées pour les modèles de Machine Learning supervisés
# La méthode describe() génère des statistiques descriptives pour les colonnes numériques sélectionnées :
# - Moyenne, écart type, valeurs minimales et maximales, quartiles.
# Ces statistiques permettent de comprendre la distribution des données pour les colonnes :
# 'recency', 'frequency', 'monetary' et les colonnes encodées des scores de review (1 à 5).

selected_columns = [
    'recency', 'frequency', 'monetary', 
    'review_score_1', 'review_score_2', 
    'review_score_3', 'review_score_4', 'review_score_5'
]

print("Résumé statistique des colonnes sélectionnées :")
print(OLIST_merged_data[selected_columns].describe())
